In [ ]:
%pip install numpy pandas scipy plotly scikit-learn lempel_ziv_complexity ordpy antropy jupytext mne PyQt6

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from scipy import signal
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import mne

from utils import json_to_edf, inspect_resting_eeg

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_colwidth', 1000)

DATADIR = Path("./data")
for fn in sorted(list(DATADIR.glob("*.edf"))): 
    print(fn.name)

In [ ]:
output = json_to_edf(
    'sensor_data.jsonl',
    'data/boundless_bob_20260225.edf',
    subject_name='bob',
    include_motion=True,
    include_ppg=True,
    line_freq=60  # Use 50 for Europe
)

In [ ]:
#file_path = DATADIR / "participant_419_eeg_raw.edf"
#file_path = DATADIR / "S01-1_annotated.edf"
file_path = DATADIR / "boundless_bob_20260225.edf"

raw = mne.io.read_raw_edf(file_path, preload=True)
print(raw.describe())

In [ ]:
inspect_resting_eeg?

In [ ]:
start_sec = 7*60
fig, raw = inspect_resting_eeg(
    'data/boundless_bob_20260225.edf', 
    eeg_channels=['AF8', 'AF7', 'TP9', 'TP10'],
    time_window=(start_sec, start_sec+30)
)
fig.savefig("bob.png")

## Eyes open/closed analysis

In [ ]:
raw = mne.io.read_raw_edf(file_path, preload=True)
print(raw.info)

print("Unique annotations:")
print(np.unique(raw.annotations.description))
print("\nAnnotation details:")
for i, (onset, duration, description) in enumerate(zip(raw.annotations.onset, raw.annotations.duration, raw.annotations.description)):
    print(f"{i}: {onset:.2f}s - {duration:.2f}s: {description}")
    if i > 20:  # Limit output
        print("... (truncated)")
        break

In [ ]:
tsplt = raw.plot(start=3, duration=120, n_channels=10)

In [ ]:
raw.plot_psd(fmax=100)

## Power Spectral Analysis: Eyes Open vs Eyes Closed


In [ ]:
# Extract epochs for each condition
# Based on annotations, create epochs for Eyes Open and Eyes Closed conditions

# Find events based on annotations
events = mne.events_from_annotations(raw)
event_dict = events[1]  # Dictionary mapping event names to codes
print("Available events:", event_dict)

# Create epochs dictionary to store different conditions
epochs_dict = {}

# Extract epochs for each condition
for condition_name, event_code in event_dict.items():
    if 'open' in condition_name.lower() or 'close' in condition_name.lower():
        # Create events array for this condition
        condition_events = events[0][events[0][:, 2] == event_code]
        
        if len(condition_events) > 0:
            # Create epochs for this condition (2 second epochs with 0.5s overlap)
            epochs = mne.Epochs(raw, condition_events, 
                              event_id={condition_name: event_code},
                              tmin=0, tmax=2.0,  # 2 second epochs
                              baseline=None,
                              preload=True,
                              verbose=False)
            epochs_dict[condition_name] = epochs
            print(f"Created {len(epochs)} epochs for condition: {condition_name}")

print(f"\nTotal conditions found: {len(epochs_dict)}")


In [ ]:
# Compute Power Spectral Density for each condition
from scipy.stats import ttest_rel
import matplotlib.pyplot as plt

# Parameters for PSD analysis
fmin, fmax = 0.5, 50  # Frequency range
alpha_band = (8, 12)  # Alpha band definition

psd_dict = {}
freqs = None

for condition_name, epochs in epochs_dict.items():
    # Compute PSD using Welch's method (updated MNE API)
    spectrum = epochs.compute_psd(method='welch', 
                                 fmin=fmin, fmax=fmax,
                                 n_fft=512,
                                 n_overlap=256,
                                 verbose=False)
    psd = spectrum.get_data()  # Get the actual PSD data
    freqs = spectrum.freqs     # Get the frequency array
    psd_dict[condition_name] = psd
    print(f"PSD computed for {condition_name}: {psd.shape}")

print(f"Frequency range: {freqs[0]:.2f} - {freqs[-1]:.2f} Hz")
print(f"Number of frequency bins: {len(freqs)}")
print(f"Number of channels: {psd.shape[1]}")


In [ ]:
# Alpha band analysis
# Extract alpha band power for each condition and channel

alpha_freq_mask = (freqs >= alpha_band[0]) & (freqs <= alpha_band[1])
alpha_power_dict = {}

for condition_name, psd in psd_dict.items():
    # Average power in alpha band (8-12 Hz) for each epoch and channel
    alpha_power = np.mean(psd[:, :, alpha_freq_mask], axis=2)  # Average over frequency
    alpha_power_dict[condition_name] = alpha_power
    print(f"Alpha power shape for {condition_name}: {alpha_power.shape}")
    print(f"Alpha power range for {condition_name}: {alpha_power.min():.2e} - {alpha_power.max():.2e}")

# Get channel names
ch_names = epochs_dict[list(epochs_dict.keys())[0]].ch_names
print(f"\nChannels: {ch_names}")


In [ ]:
# Statistical comparison between conditions
# Perform paired t-tests for alpha power between conditions

condition_names = list(alpha_power_dict.keys())
print("Statistical comparison of alpha power between conditions:")
print("=" * 60)

if len(condition_names) >= 2:
    # Compare first two conditions (should be Eyes Open vs Eyes Closed)
    cond1_name, cond2_name = condition_names[0], condition_names[1]
    cond1_alpha = alpha_power_dict[cond1_name]
    cond2_alpha = alpha_power_dict[cond2_name]
    
    # For each channel, compare alpha power between conditions
    results = []
    for ch_idx, ch_name in enumerate(ch_names):
        # Get alpha power for this channel across all epochs
        cond1_ch_alpha = cond1_alpha[:, ch_idx]
        cond2_ch_alpha = cond2_alpha[:, ch_idx]
        
        # Paired t-test (if we have paired data) or independent t-test
        from scipy.stats import ttest_ind
        t_stat, p_value = ttest_ind(cond1_ch_alpha, cond2_ch_alpha)
        
        # Calculate effect size (Cohen's d)
        pooled_std = np.sqrt(((len(cond1_ch_alpha) - 1) * np.var(cond1_ch_alpha, ddof=1) + 
                             (len(cond2_ch_alpha) - 1) * np.var(cond2_ch_alpha, ddof=1)) / 
                            (len(cond1_ch_alpha) + len(cond2_ch_alpha) - 2))
        cohens_d = (np.mean(cond1_ch_alpha) - np.mean(cond2_ch_alpha)) / pooled_std
        
        results.append({
            'channel': ch_name,
            'cond1_mean': np.mean(cond1_ch_alpha),
            'cond2_mean': np.mean(cond2_ch_alpha),
            't_stat': t_stat,
            'p_value': p_value,
            'cohens_d': cohens_d,
            'significant': p_value < 0.05
        })
        
        print(f"{ch_name:12} | {cond1_name}: {np.mean(cond1_ch_alpha):.2e} | "
              f"{cond2_name}: {np.mean(cond2_ch_alpha):.2e} | "
              f"t={t_stat:.3f}, p={p_value:.4f}, d={cohens_d:.3f} {'*' if p_value < 0.05 else ''}")
    
    # Summary
    significant_channels = [r for r in results if r['significant']]
    print(f"\nSummary:")
    print(f"Channels with significant differences: {len(significant_channels)}/{len(results)}")
    if significant_channels:
        print("Significant channels:", [r['channel'] for r in significant_channels])
        
else:
    print("Need at least 2 conditions for comparison")


In [ ]:
# Visualization: Power Spectra Comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Power Spectral Analysis: Eyes Open vs Eyes Closed', fontsize=16)

# Plot 1: Average power spectra across all channels
ax1 = axes[0, 0]
for condition_name, psd in psd_dict.items():
    # Average across epochs and channels
    mean_psd = np.mean(np.mean(psd, axis=0), axis=0)
    ax1.semilogy(freqs, mean_psd, label=condition_name, linewidth=2)

ax1.axvspan(alpha_band[0], alpha_band[1], alpha=0.3, color='gray', label='Alpha band')
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Power (μV²/Hz)')
ax1.set_title('Average Power Spectrum (All Channels)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Alpha power comparison by channel
if len(condition_names) >= 2:
    ax2 = axes[0, 1]
    cond1_alpha_mean = np.mean(alpha_power_dict[condition_names[0]], axis=0)
    cond2_alpha_mean = np.mean(alpha_power_dict[condition_names[1]], axis=0)
    
    x_pos = np.arange(len(ch_names))
    width = 0.35
    
    ax2.bar(x_pos - width/2, cond1_alpha_mean, width, label=condition_names[0], alpha=0.8)
    ax2.bar(x_pos + width/2, cond2_alpha_mean, width, label=condition_names[1], alpha=0.8)
    
    ax2.set_xlabel('Channel')
    ax2.set_ylabel('Alpha Power (μV²/Hz)')
    ax2.set_title('Alpha Power by Channel')
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(ch_names, rotation=45)
    ax2.legend()
    ax2.grid(True, alpha=0.3)

# Plot 3: Alpha power ratio (Closed/Open)
if len(condition_names) >= 2:
    ax3 = axes[1, 0]
    ratio = cond2_alpha_mean / cond1_alpha_mean
    colors = ['red' if r > 1 else 'blue' for r in ratio]
    
    bars = ax3.bar(ch_names, ratio, color=colors, alpha=0.7)
    ax3.axhline(y=1, color='black', linestyle='--', alpha=0.5)
    ax3.set_xlabel('Channel')
    ax3.set_ylabel(f'Alpha Power Ratio ({condition_names[1]}/{condition_names[0]})')
    ax3.set_title('Alpha Power Ratio by Channel')
    ax3.set_xticklabels(ch_names, rotation=45)
    ax3.grid(True, alpha=0.3)
    
    # Add text annotations for significant differences
    for i, (bar, result) in enumerate(zip(bars, results)):
        if result['significant']:
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                    '*', ha='center', va='bottom', fontweight='bold', fontsize=12)

# Plot 4: Topographic representation (if channels are available)
ax4 = axes[1, 1]
if len(condition_names) >= 2:
    # Simple bar plot showing effect sizes
    effect_sizes = [r['cohens_d'] for r in results]
    p_values = [r['p_value'] for r in results]
    colors = ['red' if p < 0.05 else 'gray' for p in p_values]
    
    bars = ax4.bar(ch_names, effect_sizes, color=colors, alpha=0.7)
    ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    ax4.set_xlabel('Channel')
    ax4.set_ylabel("Cohen's d (Effect Size)")
    ax4.set_title('Effect Size by Channel (Red = Significant)')
    ax4.set_xticklabels(ch_names, rotation=45)
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Additional Analysis: Frequency Band Comparison
# Compare multiple frequency bands between conditions

frequency_bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 30),
    'Gamma': (30, 50)
}

print("Frequency Band Analysis:")
print("=" * 80)

if len(condition_names) >= 2:
    band_results = {}
    
    for band_name, (f_low, f_high) in frequency_bands.items():
        band_mask = (freqs >= f_low) & (freqs <= f_high)
        band_power_dict = {}
        
        for condition_name, psd in psd_dict.items():
            # Average power in this frequency band
            band_power = np.mean(psd[:, :, band_mask], axis=2)  # Average over frequency
            band_power_dict[condition_name] = band_power
        
        # Statistical comparison for this band
        cond1_band = band_power_dict[condition_names[0]]
        cond2_band = band_power_dict[condition_names[1]]
        
        # Average across all channels and epochs
        cond1_mean = np.mean(cond1_band)
        cond2_mean = np.mean(cond2_band)
        
        # T-test across all data points
        cond1_flat = cond1_band.flatten()
        cond2_flat = cond2_band.flatten()
        t_stat, p_value = ttest_ind(cond1_flat, cond2_flat)
        
        # Effect size
        pooled_std = np.sqrt(((len(cond1_flat) - 1) * np.var(cond1_flat, ddof=1) + 
                             (len(cond2_flat) - 1) * np.var(cond2_flat, ddof=1)) / 
                            (len(cond1_flat) + len(cond2_flat) - 2))
        cohens_d = (cond1_mean - cond2_mean) / pooled_std
        
        band_results[band_name] = {
            'cond1_mean': cond1_mean,
            'cond2_mean': cond2_mean,
            'ratio': cond1_mean / cond2_mean,
            't_stat': t_stat,
            'p_value': p_value,
            'cohens_d': cohens_d,
            'significant': p_value < 0.05
        }
        
        print(f"{band_name:8} ({f_low:2.0f}-{f_high:2.0f} Hz) | "
              f"{condition_names[0]}: {cond1_mean:.2e} | "
              f"{condition_names[1]}: {cond2_mean:.2e} | "
              f"Ratio: {cond1_mean/cond2_mean:.3f} | "
              f"t={t_stat:.3f}, p={p_value:.4f}, d={cohens_d:.3f} "
              f"{'***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else ''}")
    
    print(f"\n*** p < 0.001, ** p < 0.01, * p < 0.05")
    
    # Summary of significant bands
    significant_bands = [band for band, result in band_results.items() if result['significant']]
    if significant_bands:
        print(f"\nSignificant frequency bands: {', '.join(significant_bands)}")
        
        # Alpha band specific findings
        if 'Alpha' in significant_bands:
            alpha_result = band_results['Alpha']
            print(f"\nAlpha Band Findings:")
            print(f"- Alpha power is {alpha_result['ratio']:.2f}x higher in {condition_names[0]} vs {condition_names[1]}")
            print(f"- Effect size (Cohen's d): {alpha_result['cohens_d']:.3f}")
            print(f"- Statistical significance: p = {alpha_result['p_value']:.4f}")
            
            if alpha_result['ratio'] > 1:
                print(f"- This suggests increased alpha activity during {condition_names[0]}")
            else:
                print(f"- This suggests decreased alpha activity during {condition_names[0]}")
    else:
        print("\nNo significant differences found in any frequency band.")
        
else:
    print("Need at least 2 conditions for comparison")


## Interactive EEG Plotting


In [ ]:
# Configure interactive plotting for MNE
# Option 1: Use Qt backend for native MNE interactive plots
%matplotlib qt

# Alternative: If you prefer plots within the notebook, use:
# %matplotlib widget  

# Set MNE to use the interactive browser backend
import matplotlib
matplotlib.use('Qt5Agg')  # or 'TkAgg' if Qt is not available

print("Interactive plotting configured!")
print("Backend:", matplotlib.get_backend())


In [ ]:
# Now create an interactive plot of the raw EEG data
# This will open in a separate window with full MNE interactive capabilities

# Interactive raw data browser
fig = raw.plot(
    start=0,           # Start time in seconds
    duration=30,       # Duration to show in seconds  
    n_channels=20,     # Number of channels to show at once
    scalings='auto',   # Auto-scale channels
    title='Interactive EEG Browser - Eyes Open vs Eyes Closed',
    show=True,         # Show the plot
    block=False        # Don't block execution
)

print("Interactive plot opened in separate window!")
print("Use the following controls:")
print("- Left/Right arrows: Navigate time")
print("- Up/Down arrows: Change amplitude scaling") 
print("- Page Up/Down: Navigate channels")
print("- 'a': Toggle annotation mode")
print("- Space: Start/stop scrolling")
print("- 'h': Help menu")
print("\nYou can see the Eyes Open/Closed annotations marked on the timeline!")

In [ ]:
# Alternative: Plotly-based interactive plot (stays in notebook)
# If the Qt backend doesn't work, here's a Plotly alternative

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_interactive_eeg_plot(raw, start_time=0, duration=30, channels_to_show=None):
    """Create an interactive EEG plot using Plotly"""
    
    if channels_to_show is None:
        # Show all channels except ref and drl
        eeg_channels = [ch for ch in raw.ch_names if ch.startswith('eeg')]
        channels_to_show = eeg_channels
    
    # Get data for the specified time window
    start_sample = int(start_time * raw.info['sfreq'])
    end_sample = int((start_time + duration) * raw.info['sfreq'])

    data, times = raw[:, start_sample:end_sample]
    #times = times + start_time  # Adjust time offset
    #print(f"{start_sample=}, {end_sample=}, {duration=}, start_time={times[0]}, end_time={times[-1]}")
    
    # Create subplot for each channel
    fig = make_subplots(
        rows=len(channels_to_show), 
        cols=1,
        shared_xaxes=True,
        subplot_titles=[f"Channel {ch}" for ch in channels_to_show],
        vertical_spacing=0.02
    )
    
    # Add traces for each channel
    for i, ch_name in enumerate(channels_to_show):
        if ch_name in raw.ch_names:
            ch_idx = raw.ch_names.index(ch_name)
            y_data = data[ch_idx, :] * 1e6  # Convert to microvolts
            
            fig.add_trace(
                go.Scatter(
                    x=times,
                    y=y_data,
                    mode='lines',
                    name=ch_name,
                    line=dict(width=1),
                    showlegend=False
                ),
                row=i+1, col=1
            )
    
    # Add annotations for Eyes Open/Closed periods
    for onset, duration_ann, description in zip(raw.annotations.onset, 
                                               raw.annotations.duration, 
                                               raw.annotations.description):
        if onset >= start_time and onset <= start_time + duration:
            color = 'lightblue' if 'Open' in description else 'lightcoral'
            fig.add_vrect(
                x0=onset, x1=onset + duration_ann,
                fillcolor=color, opacity=0.3,
                annotation_text=description,
                annotation_position="top left"
            )
    
    # Update layout
    fig.update_layout(
        title=f'EEG Timeseries',
        height=120 * len(channels_to_show),
        xaxis_title='Time (s)',
        showlegend=False,
        hovermode='x unified'
    )
    
    # Update y-axes
    for i in range(len(channels_to_show)):
        fig.update_yaxes(title_text='μV', row=i+1, col=1)
    
    return fig

# Create the interactive plot
interactive_fig = create_interactive_eeg_plot(raw, start_time=10, duration=110)
interactive_fig.show()


In [ ]:
# Interactive widget for exploring different time windows
# Create a simple widget to change time windows easily

def plot_time_window(start_time=0, duration=30, n_channels=8):
    """Function to plot different time windows"""
    fig = create_interactive_eeg_plot(raw, start_time, duration, 
                                    channels_to_show=raw.ch_names[:n_channels])
    fig.show()
    
# You can call this function to explore different time windows:
print("Use this function to explore different time windows:")
print("plot_time_window(start_time=60, duration=30)  # Show 60-90s")
print("plot_time_window(start_time=120, duration=45) # Show 120-165s")
print("plot_time_window(start_time=180, duration=60) # Show 180-240s")
print()
print("Or modify parameters:")
print("plot_time_window(start_time=0, duration=120, n_channels=12)  # Longer window, more channels")

# Example: Show the transition between first Eyes Open and Eyes Closed periods
print("\nShowing transition from Eyes Open to Eyes Closed (around 60s):")
transition_fig = create_interactive_eeg_plot(raw, start_time=50, duration=30)
transition_fig.show()


In [ ]:

from scipy.io import loadmat
import plotly.express as px

d = loadmat("/Users/bobd/git/eeg-recorder/analysis/data/ding_2025/data.mat")
evoke = d['evoke']
print(f"{evoke.shape=}")
itpc = d['itpc']
print(f"{itpc.shape=}")

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats

# Calculate mean across trials and participants for channel -2
evoke_mn = evoke.mean(axis=2).mean(axis=0)
freq = np.linspace(1, 30, evoke_mn.shape[1])

# Calculate 95% confidence intervals
# First get data for channel -2 across all trials and participants
channel_data = evoke[:, :, :, -2]  # shape: (participants, channels, trials)
# Average across channels (axis=1) to get (participants, trials) 
channel_data_avg = channel_data.mean(axis=1)  # shape: (participants, trials)
# Now we have data points across participants and trials for statistical analysis

# Calculate mean and standard error across all data points
all_freq_data = []
for freq_idx in range(evoke_mn.shape[1]):
    # Get all data points for this frequency (across participants and trials)
    freq_data = evoke[:, :, :, freq_idx].mean(axis=1)  # Average across channels first
    freq_data_flat = freq_data.flatten()  # Flatten to get all data points
    all_freq_data.append(freq_data_flat)

# Calculate mean, standard error, and 95% CI for each frequency
means = []
ci_lower = []
ci_upper = []

for freq_data in all_freq_data:
    mean_val = np.mean(freq_data)
    sem = stats.sem(freq_data)  # Standard error of the mean
    ci = stats.t.interval(0.95, len(freq_data)-1, loc=mean_val, scale=sem)
    
    means.append(mean_val)
    ci_lower.append(ci[0])
    ci_upper.append(ci[1])

means = np.array(means)
ci_lower = np.array(ci_lower)
ci_upper = np.array(ci_upper)

# Create plot with confidence intervals
fig = go.Figure()

# Add the main line
fig.add_trace(go.Scatter(
    x=freq,
    y=means,
    mode='lines',
    name='Mean',
    line=dict(color='blue', width=2)
))

# Add confidence interval as filled area
fig.add_trace(go.Scatter(
    x=np.concatenate([freq, freq[::-1]]),  # x coordinates for filled area
    y=np.concatenate([ci_upper, ci_lower[::-1]]),  # y coordinates for filled area
    fill='toself',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='95% CI',
    showlegend=True
))

fig.update_layout(
    title='Evoked Response with 95% Confidence Intervals',
    xaxis_title='Frequency (Hz)',
    yaxis_title='Amplitude',
    hovermode='x unified'
)

fig.show()


In [ ]:
itpc_mn = itpc.mean(axis=2).mean(axis=0)
px.line(x=freq, y=itpc_mn[-1, :])